In [75]:
import numpy as np
import pandas as pd
import re

In [23]:
raw_data = pd.read_csv('theoffice.csv')
len(raw_data)

59909

In [26]:
data = raw_data[raw_data['deleted'] == False] # remove deleted scenes
len(data)

57973

In [43]:
unique_speakers = data['speaker'].unique()
Dw = []
Jim = []
for speaker in unique_speakers:
    if "Dw" in speaker:
        Dw.append(speaker)
    if "Ji" in speaker:
        Jim.append(speaker)
print(Dw)
print()
print(Jim)

['Dwight', 'Michael and Dwight', 'Dwight and Michael', 'Jim and Dwight', 'Jim, Josh, and Dwight', 'Dwight ', 'Dwight:', 'Michael & Dwight', 'DwightKSchrute', 'Michael/Dwight', 'Angela and Dwight', 'Jim, Pam, Michael and Dwight', 'Dwight and Erin', 'Dwight and Andy', 'Dwight & Andy', 'Dwight, Jim & Pam', 'Andy and Dwight', 'Dwight, Jim and Michael', 'Dwight.', 'Dwight and Angela', 'Employees except Dwight', 'Dwight & Nate', 'Phyllis, Stanley, Dwight', 'Dwight [on phone]', 'Dwight, Erin, Jim & Kevin', 'Pam & Dwight', 'Jim & Dwight']

['Jim', 'Pam and Jim', 'Michael and Jim', 'Jim and Pam', "Jim's voicemail", 'Jim and Dwight', 'Pam/Jim', 'Jim, Josh, and Dwight', 'Andy and Jim', 'Jim ', 'Jim, Pam, Michael and Dwight', 'Dwight, Jim & Pam', 'Jim & Pam', 'Dwight, Jim and Michael', "Jim's Dad", 'Dwight, Erin, Jim & Kevin', 'Fake Jim', 'Jim & Dwight']


In [57]:
JD_list = ['Jim', 'Dwight', 'Dwight.', 'DwightKSchrute', 'Dwight ','Dwight:', 'Dwight[on phone]', 'Jim '] 
Jim_Dwight_data = data[data.speaker.isin(JD_list)] # dataframe of only jim and dwight lines

In [69]:
Jim_Dwight_data.sport = Jim_Dwight_data.speaker.apply(lambda x: 'Dwight' if 'Dw' in x else x) # clean up names
Jim_Dwight_data.sport = Jim_Dwight_data.speaker.apply(lambda x: 'Jim' if 'Jim' in x else x)

Jim_Dwight_data['speaker'].unique()

array(['Jim', 'Dwight'], dtype=object)

In [89]:
# clean up messy data
Jim_Dwight_data['line_text'].str.replace("���", "'")

1               Oh, I told you. I couldn't close it. So...
3               Actually, you called me in here, but yeah.
16       [singing] Shall I play for you? Pa rum pump um...
17       My job is to speak to clients on the phone abo...
19           Whassup! I still love that after seven years.
21                                                Whassup!
23                                                Whassup.
25                                                Nothing.
27                                   All right. Take care.
70                                                    Yes?
72                         Yeah, stop. That is ridiculous.
84       Sure. Mr. Davis, let me call you right back. Y...
85                                                   What?
86                                     What are you doing?
87             Just clearing my desk. I can't concentrate.
88                                  It's not on your desk.
89       It's overlapping. It's all spilling over the e.

In [90]:
# determine if it is a convo between Jim and Dwight: if it is the same scene, same episode and id is consecutive
Jim_Dwight_data.drop(['deleted'], axis=1)
Jim_Dwight_data.columns = ['id', 'season', 'episode', 'scene', 'line_text', 'speaker', 'is_convo']

ValueError: labels ['deleted'] not contained in axis

In [99]:
Jim_Dwight_data.head()

,id,season,episode,scene,line_text,speaker,is_convo
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False
16,17,1,1,5,[singing] Shall I play for you? Pa rum pump um...,Dwight,False
17,18,1,1,6,My job is to speak to clients on the phone abo...,Jim,False
19,20,1,1,7,Whassup! I still love that after seven years.,Jim,False


In [127]:
# if curr line has the same scene, consecutive id and different speaker with prev line, copy prev line's is_convo id
def isJDconvo(df, idx):
    if df.loc[i, 'speaker'] == "Jim" and df.loc[i-1, 'speaker'] == "Dwight":
        if df.loc[i-1, 'scene'] == df.loc[i, 'scene']:
            if df.loc[i-1, 'index'] == df.loc[i, 'index'] - 1:
                return True
    elif df.loc[i, 'speaker'] == "Dwight" and df.loc[i-1, 'speaker'] == "Jim":
        if df.loc[i-1, 'scene'] == df.loc[i, 'scene']:
            if df.loc[i-1, 'index'] == df.loc[i, 'index'] - 1:
                return True
    return False

In [128]:
jd_data = Jim_Dwight_data.copy()
jd_data['is_convo'] = 0
jd_data.reset_index(inplace=True)
pair_code = 0
for i in range(len(jd_data)):
    if i == 0:
        jd_data.loc[0, 'is_convo'] = pair_code
        pair_code = pair_code + 1
    else:
        if isJDconvo(jd_data, i):
            jd_data.loc[i, 'is_convo'] = jd_data.loc[i-1, 'is_convo']
        else:
            jd_data.loc[i, 'is_convo'] = pair_code
            pair_code = pair_code + 1
        

In [134]:
jd_data[jd_data.duplicated('is_convo',keep=False)]


,index,id,season,episode,scene,line_text,speaker,is_convo
11,84,85,1,1,19,"Sure. Mr. Davis, let me call you right back. Y...",Jim,11
12,85,86,1,1,19,What?,Dwight,11
13,86,87,1,1,19,What are you doing?,Jim,11
14,87,88,1,1,19,Just clearing my desk. I can't concentrate.,Dwight,11
15,88,89,1,1,19,It's not on your desk.,Jim,11
16,89,90,1,1,19,It's overlapping. It's all spilling over the e...,Dwight,11
17,90,91,1,1,19,You can't do that.,Dwight,12
18,91,92,1,1,19,Why not?,Jim,12
19,92,93,1,1,19,Safety violation. I could fall and pierce an o...,Dwight,12
20,93,94,1,1,19,[crosses fingers] We'll see. [Dwight begins sm...,Jim,12
